In [1]:
import sys
import os
import subprocess

print("Python executable:", sys.executable)
print("Python prefix:", sys.prefix)
print("Conda environment:", os.environ.get('CONDA_DEFAULT_ENV'))
print("Conda prefix:", os.environ.get('CONDA_PREFIX'))

# Check pip location
result = subprocess.run(['which', 'pip'], capture_output=True, text=True)
print("Pip location:", result.stdout.strip())

# Check pip version
result = subprocess.run(['pip', '--version'], capture_output=True, text=True)
print("Pip version:", result.stdout.strip())

Python executable: /home/sagemaker-user/.conda/envs/inception/bin/python
Python prefix: /home/sagemaker-user/.conda/envs/inception
Conda environment: base
Conda prefix: /opt/conda
Pip location: /opt/conda/bin/pip
Pip version: pip 25.1.1 from /opt/conda/lib/python3.12/site-packages/pip (python 3.12)


In [2]:
import os
import sys

# Add environment's bin directory to PATH
env_bin = os.path.dirname(sys.executable)
current_path = os.environ['PATH']
if env_bin not in current_path:
    os.environ['PATH'] = f"{env_bin}:{current_path}"

# Verify fix
result = subprocess.run(['which', 'pip'], capture_output=True, text=True)
print("Fixed pip location:", result.stdout.strip())

Fixed pip location: /home/sagemaker-user/.conda/envs/inception/bin/pip


# inceptionai/jais-adapted-7b

In [ ]:
# -*- coding: utf-8 -*-

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "inceptionai/jais-adapted-7b"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", trust_remote_code=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def get_response(text, tokenizer=tokenizer, model=model):
    tokenized = tokenizer(text, return_tensors="pt")
    input_ids, attention_mask = tokenized['input_ids'].to(device), tokenized['attention_mask'].to(device)
    input_len = input_ids.shape[-1]
    generate_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        top_p=0.9,
        temperature=0.3,
        max_length=2048,
        min_length=input_len + 4,
        repetition_penalty=1.2,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id
    )
    response = tokenizer.batch_decode(
        generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )[0]
    return response

text = "عاصمة دولة الإمارات العربية المتحدة ه"
print(get_response(text))

text = "The capital of UAE is"
print(get_response(text))
